In [1]:
import mne
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from scipy.fft import fftshift
# from compute_abr import *
from bids import BIDSLayout

In [26]:
BIDS_ROOT = '../data/bids'
FS = 5000
stim_freqs = [130, 200, 280]

cond_dict = {
    (130, 1, 'l', True): ['211', '212', '213'],
    (200, 2, 'l', True): ['221', '222', '223'],
    (280, 3, 'l', True): ['231', '232', '233'],
    (130, 1, 'r', True): ['111', '121', '131'],
    (200, 2, 'r', True): ['112', '122', '132'],
    (280, 3, 'r', True): ['113', '123', '133'],
    (130, 1, 'l', False): ['111', '112', '113'],
    (200, 2, 'l', False): ['121', '122', '123'],
    (280, 3, 'l', False): ['131', '132', '133'],
    (130, 1, 'r', False): ['211', '221', '231'],
    (200, 2, 'r', False): ['212', '222', '232'],
    (280, 3, 'r', False): ['213', '223', '233']
}

right_channels = ['AF4','C2','C4','C6','T8','CP2','CP4','CP6','F2','F4','F6','F8','FC2','FC4','FC6','FT10','FT8','Fp2','O2','P2','P4','P6','P8','PO4','PO8','TP10','TP8']
left_channels = ['AF3', 'C1','C3','C5','T7','CP1','CP3','CP5','F1','F3','F5','F7','FC1','FC3','FC5','FT7','FT9','Fp1','O1','P1','P3','P5','P7','PO3','PO7','TP7','TP9']


In [3]:
def read_epochs(sub, desc):
    '''
    reads and concatenates epochs across runs
    vendored from John
    '''
    layout = BIDSLayout(BIDS_ROOT, derivatives = True)
    run = lambda f: int(re.findall('run-(\w+)_', f)[0])
    fnames = layout.get(
        return_type = 'filename',
        subject = sub, 
        desc = desc
        )
    fnames.sort(key = run)
    epochs_all = [mne.read_epochs(f) for f in fnames]
    epochs = mne.concatenate_epochs(epochs_all)
    epochs = epochs.pick('eeg')
    return epochs

def get_dB(epochs):
    P = epochs.average().compute_psd(tmin = 0, tmax = 0.4, fmin = 100, fmax = 300)
    P0 = epochs.average().compute_psd(tmin = -0.4, tmax = 0, fmin = 100, fmax = 300)
    dB = 10*np.log10(P.get_data()/P0.get_data())
    return dB, P.freqs

def get_hemisphere(ch_names):
    left_channels = ['AF3', 'C1','C3','C5','T7','CP1','CP3','CP5','F1','F3','F5','F7','FC1','FC3','FC5','FT7','FT9','Fp1','O1','P1','P3','P5','P7','PO3','PO7','TP7','TP9']
    right_channels = ['AF4','C2','C4','C6','T8','CP2','CP4','CP6','F2','F4','F6','F8','FC2','FC4','FC6','FT10','FT8','Fp2','O2','P2','P4','P6','P8','PO4','PO8','TP10','TP8']
    hemisphere = []
    for channel in epochs.ch_names:
        if channel in left_channels:
            hemisphere.append("left")
        elif channel in right_channels:
            hemisphere.append("right")
        else:
            hemisphere.append("middle")
    return hemisphere

def get_hemisphere_ch_names(ch_names, hemisphere_channels):
    hemisphere_ch_names = set(ch_names).intersection(set(hemisphere_channels))
    return list(hemisphere_ch_names)

def get_channel_coords(coords, channel_names):
    x_coords = []
    y_coords = []
    for channel in channel_names:
        x_coord = coords['x'][coords['ch_name'] == channel]
        x_coord = x_coord.tolist()[0]
        x_coords.append(x_coord)
        y_coord = coords['y'][coords['ch_name'] == channel]
        y_coord = y_coord.tolist()[0]
        y_coords.append(y_coord)

    return(x_coords, y_coords)

def plot_dB(ax, dB, freqs, attend, stim_freq):
#     print(str(condition))
    ax.plot(freqs, dB, label = attend)
    ax.set_xlabel('frequency')
    ax.set_ylabel('dB')
    ax.axvline(stim_freq, linestyle = '--', color = 'grey')
    ax.legend(title = 'Attend')

In [7]:
coords = pd.read_csv('../data/captrak/2dcoords.csv')
layout = BIDSLayout(BIDS_ROOT, derivatives = True)
subs = layout.get_subjects(scope = 'preprocessing')
all_spectrums = []

for sub in subs:
    # Load data
    epochs = read_epochs(sub, 'clean')
    
    # Iterate over conditions
    for condition, event_ids in cond_dict.items():
        ch_names = epochs.ch_names
        n_chans = len(ch_names)
        stim_freq = condition[0]
    
        # Compute dB for each channel
        cond_epochs = epochs[event_ids]
        dB, psd_freqs = get_dB(cond_epochs) # dB should be n_chans * n_freq
        
        # Iterate over stim freqs
        for freq in freqs:
        
            # select power at stimulus frequency
            idx = np.abs(psd_freqs - freq).argmin()
            freq_dB = dB[:,idx]

            # Name channels by left or right hemisphere
            hemisphere = get_hemisphere(epochs.ch_names)

            # Get channel coordinates
            x, y = get_channel_coords(coords, ch_names)

            # Create data frame (channel name, hemisphere, stim_freq, stream, attended, dB)
            df = pd.DataFrame(
                {'ch_names': ch_names,
                 'hemisphere': hemisphere,
                 'x': x,
                 'y': y,
                 'freq': freq,
                 'stim_freq': stim_freq,
                 'stream': condition[2],
                 'attended': condition[3],
                 'dB': freq_dB
                }
            )
            all_spectrums.append(df)

spectrums = pd.concat(all_spectrums)
spectrums.to_csv('spectrums.csv', sep = '\t', index = False)

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


Reading /project2/hcn1/dichotic_pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-4/sub-4_task-dichotic_run-1_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -400.00 ...     450.00 ms
        0 CTF compensation matrices available
Not setting metadata
1465 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_24109/456505877.py:15: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_all)


Not setting metadata
1465 matching events found
Applying baseline correction (mode: mean)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
(61, 80)
(61, 80)
(61, 80)
(61, 80)
(61, 80)
(61, 80)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
(61, 80)
(61, 80)
(61, 80)
(61, 80)
(61, 80)
(61, 80)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multita

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


...

Not setting metadata
1765 matching events found
Applying baseline correction (mode: mean)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multita

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


Reading /project2/hcn1/dichotic_pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-6/sub-6_task-dichotic_run-1_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -400.00 ...     450.00 ms
        0 CTF compensation matrices available
Not setting metadata
1858 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_24109/456505877.py:15: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_all)


Not setting metadata
1858 matching events found
Applying baseline correction (mode: mean)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
(62, 80)
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multita

## Plot

In [6]:
spectrums = pd.read_csv('spectrums.csv', sep = '\t')
spectrums

,ch_names,hemisphere,x,y,stim_freq,stream,attended,dB
0,Fp1,left,174,397,130,l,True,1.680979
1,Fz,middle,224,325,130,l,True,0.969817
2,F3,left,156,328,130,l,True,0.595695
3,F7,left,94,339,130,l,True,2.710856
4,FT9,left,32,306,130,l,True,-4.046504
...,...,...,...,...,...,...,...,...
14863,F6,right,324,331,280,r,False,-0.548623
14864,AF4,right,273,366,280,r,False,-0.738886
14865,F2,right,257,326,280,r,False,0.065810
14866,FCz,middle,224,286,280,r,False,1.883340


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
graph = sns.lineplot(data = spectrums, x = "frequency", y = "power", hue = "stimulus", palette = 'tab10', ax = ax)
# graph.axvline(50, linestyle = 'dashed', color = 'blue')
# graph.axvline(100, linestyle = 'dashed', color = 'orange')
# graph.axvline(150, linestyle = 'dashed', color = 'green')
# graph.axvline(200, linestyle = 'dashed', color = 'red')
# graph.axvline(250, linestyle = 'dashed', color = 'purple')
graph.legend(title = 'Stim freq (Hz)')
graph.set_ylabel("Power (dB)")
graph.set_xlabel("Frequency (Hz)")
graph.set_xlim(0, 350)

### Compute spectrum by hemisphere

In [14]:
epochs

Number of events,1465
Events,111: 58112: 86113: 97121: 61122: 58123: 83131: 70132: 73133: 71211: 62212: 112213: 96221: 112222: 89223: 109231: 82232: 81233: 65
Time range,-0.400 – 0.450 s
Baseline,-0.400 – 0.000 s


In [4]:
def get_hemisphere_channels(ch_names):
    left_channels = ['AF3', 'C1','C3','C5','T7','CP1','CP3','CP5','F1','F3','F5','F7','FC1','FC3','FC5','FT7','FT9','Fp1','O1','P1','P3','P5','P7','PO3','PO7','TP7','TP9']
    right_channels = ['AF4','C2','C4','C6','T8','CP2','CP4','CP6','F2','F4','F6','F8','FC2','FC4','FC6','FT10','FT8','Fp2','O2','P2','P4','P6','P8','PO4','PO8','TP10','TP8']

    # Find channels that are missing
    missing = set(left_channels + right_channels) - set(ch_names)
    print(missing)

    # Pop from both hemispheres
    for channel in missing:
        if channel in left_channels:
            print('left')
            i = left_channels.index(channel)
        elif channel in right_channels:
            print('right')
            i = right_channels.index(channel)
        right_channels.pop(i)
        left_channels.pop(i)

    print(right_channels)
    print(left_channels)
    return left_channels, right_channels

In [38]:
coords = pd.read_csv('../data/captrak/2dcoords.csv')
layout = BIDSLayout(BIDS_ROOT, derivatives = True)
subs = layout.get_subjects(scope = 'preprocessing')
diff_spectrums = []

for sub in subs:
    # Load data
    epochs = read_epochs(sub, 'clean')
    
    # Iterate over conditions
    for condition, event_ids in cond_dict.items():
        stim = condition[0]
        
        # Pick channels for each hemisphere
        left_channels, right_channels = get_hemisphere_channels(epochs.ch_names)
        left_epochs = epochs[event_ids].pick(left_channels)
        right_epochs = epochs[event_ids].pick(right_channels)

        # Compute dB for each channel
        left_dB, psd_freqs = get_dB(left_epochs) # dB should be n_chans * n_freq
        right_dB, psd_freqs = get_dB(right_epochs) # preserves the requested channel order given as its picks parameter
        dB_diff = left_dB - right_dB
        
        # Iterate over stim freqs
        for freq in stim_freqs:
        
            # select Fourier frequencies around stimulus frequency
            idx = np.abs(psd_freqs - freq).argmin()
            freq_dB_diff = dB_diff[:,idx]

            # Get channel coordinates
            x, y = get_channel_coords(coords, left_channels)

            # Create data frame (channel name, hemisphere, stim_freq, stream, attended, dB)
            df = pd.DataFrame(
                {'left_chs': left_channels,
                 'x': x,
                 'y': y,
                 'freq': freq,
                 'stim': stim,
                 'stream': condition[2],
                 'attended': condition[3],
                 'dB_diff': freq_dB_diff
                }
            )
            diff_spectrums.append(df)

diff_spectrums = pd.concat(diff_spectrums)
diff_spectrums.to_csv('diff_spectrums.csv', sep = '\t', index = False)

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


Reading /project2/hcn1/dichotic_pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-43/sub-43_task-dichotic_run-1_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -400.00 ...     450.00 ms
        0 CTF compensation matrices available
Not setting metadata
1782 matching events found
No baseline correction applied
0 projection items activated


/tmp/ipykernel_19919/456505877.py:15: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_all)


Not setting metadata
1782 matching events found
Applying baseline correction (mode: mean)
set()
['AF4', 'C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT10', 'FT8', 'Fp2', 'O2', 'P2', 'P4', 'P6', 'P8', 'PO4', 'PO8', 'TP10', 'TP8']
['AF3', 'C1', 'C3', 'C5', 'T7', 'CP1', 'CP3', 'CP5', 'F1', 'F3', 'F5', 'F7', 'FC1', 'FC3', 'FC5', 'FT7', 'FT9', 'Fp1', 'O1', 'P1', 'P3', 'P5', 'P7', 'PO3', 'PO7', 'TP7', 'TP9']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with

    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
set()
['AF4', 'C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT10', 'FT8', 'Fp2', 'O2', 'P2', 'P4', 'P6', 'P8', 'PO4', 'PO8', 'TP10', 'TP8']
['AF3', 'C1', 'C3', 'C5', 'T7', 'CP1', 'CP3', 'CP5', 'F1', 'F3', 'F5', 'F7', 'FC1', 'FC3', 'FC5', 'FT7', 'FT9', 'Fp1', 'O1', 'P1', 'P3', 'P5', 'P7', 'PO3', 'PO7', 'TP7', 'TP9']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_ch

...

/tmp/ipykernel_19919/456505877.py:15: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs(epochs_all)


Not setting metadata
1846 matching events found
Applying baseline correction (mode: mean)
set()
['AF4', 'C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT10', 'FT8', 'Fp2', 'O2', 'P2', 'P4', 'P6', 'P8', 'PO4', 'PO8', 'TP10', 'TP8']
['AF3', 'C1', 'C3', 'C5', 'T7', 'CP1', 'CP3', 'CP5', 'F1', 'F3', 'F5', 'F7', 'FC1', 'FC3', 'FC5', 'FT7', 'FT9', 'Fp1', 'O1', 'P1', 'P3', 'P5', 'P7', 'PO3', 'PO7', 'TP7', 'TP9']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with

    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
set()
['AF4', 'C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT10', 'FT8', 'Fp2', 'O2', 'P2', 'P4', 'P6', 'P8', 'PO4', 'PO8', 'TP10', 'TP8']
['AF3', 'C1', 'C3', 'C5', 'T7', 'CP1', 'CP3', 'CP5', 'F1', 'F3', 'F5', 'F7', 'FC1', 'FC3', 'FC5', 'FT7', 'FT9', 'Fp1', 'O1', 'P1', 'P3', 'P5', 'P7', 'PO3', 'PO7', 'TP7', 'TP9']
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
    Using multitaper spectrum estimation with 7 DPSS windows
NOTE: pick_ch

/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/bids/layout/validation.py:151: UserWarning: The PipelineDescription field was superseded by GeneratedBy in BIDS 1.4.0. You can use ``pybids upgrade`` to update your derivative dataset.
  warnings.warn("The PipelineDescription field was superseded "


Reading /project2/hcn1/dichotic_pitch_tracking/analysis/../data/bids/derivatives/preprocessing/sub-14/sub-14_task-dichotic_run-1_desc-clean_epo.fif.gz ...


In [40]:
df